In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/preprocessed-dallas-public-safety/filtered_data_3.csv


In [2]:
import pandas as pd
import numpy as np
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

In [3]:
# Load your dataset
data = pd.read_csv('/kaggle/input/preprocessed-dallas-public-safety/filtered_data_3.csv')

In [4]:
data = data.dropna()


In [5]:
data.head

<bound method NDFrame.head of          Zip Code Day1 of the Week Time1 of Occurrence       Time Bin  \
0         75231.0              Tue               20:00          Night   
1         75240.0              Fri               12:00      Afternoon   
2         75215.0              Mon               12:00      Afternoon   
3         75209.0              Thu               01:38  Early Morning   
4         75208.0              Mon               17:30      Afternoon   
...           ...              ...                 ...            ...   
1127417   75205.0              Sat               10:45        Morning   
1127418   75204.0              Mon               19:00          Night   
1127419   75225.0              Sat               17:45      Afternoon   
1127420   75228.0              Tue               22:30          Night   
1127421   75238.0              Tue               12:30      Afternoon   

              Division  Sector  Zip-Time-Percentage  Zipcode-Day-Percentage  \
0            N

In [6]:
# Define the features for clustering
features_for_clustering = ['Zip Code', 'Time Bin', 'Day1 of the Week', 
                           'Zipcode-Percentage',
                           'Zipcode-Day-Percentage',
                           'Zip-Time-Percentage',
                           'Incident_Score']


In [7]:
# Separate numerical and categorical features
numeric_features = ['Zipcode-Percentage',
                           'Zipcode-Day-Percentage',
                           'Zip-Time-Percentage',
                           'Incident_Score']

In [8]:
categorical_features = ['Zip Code', 'Time Bin', 'Day1 of the Week']


In [9]:
# Create transformers for numeric and categorical features
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(drop='first')

In [10]:
# Create a preprocessor that applies the transformers
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [11]:
# Fit the preprocessor to the data and transform the features
data_encoded_normalized = preprocessor.fit_transform(data[features_for_clustering])


In [12]:

# Create a Gaussian Mixture Model with the desired number of components (clusters)
n_components = 5  # You can adjust this
gmm = GaussianMixture(n_components=n_components, random_state=42)

In [13]:
# Fit the GMM model and get cluster assignments
data['Cluster'] = gmm.fit_predict(data_encoded_normalized.toarray())

In [14]:
# Create a function to sample Safety Scores from the cluster's distribution
def sample_safety_score(cluster_mean, cluster_cov):
    return np.random.normal(cluster_mean, np.sqrt(cluster_cov))

In [15]:
# Iterate through each cluster to assign Safety Scores
for cluster_id, cluster_data in data.groupby('Cluster'):
    cluster_mean = cluster_data['Incident_Score'].mean()
    cluster_cov = cluster_data['Incident_Score'].var()
    data.loc[data['Cluster'] == cluster_id, 'Predicted Safety Score'] = cluster_data.apply(lambda row: sample_safety_score(cluster_mean, cluster_cov), axis=1)

# Create a Min-Max scaler
scaler = MinMaxScaler(feature_range=(0, 100))

# Fit and transform the 'Predicted Safety Score' column
data['Predicted Safety Score'] = scaler.fit_transform(data['Predicted Safety Score'].values.reshape(-1, 1))


In [16]:
# Save the updated dataset with the predicted Safety Scores
data.to_csv('/kaggle/working/dataset_with_safety_scores.csv', index=False)

In [17]:
# Define the target variable and features for your model
X = data_encoded_normalized
y = data['Predicted Safety Score']

# Create your Random Forest Regressor model
model = RandomForestRegressor()

# Perform cross-validation (5-fold cross-validation)
scores = cross_val_score(model, X, y, cv=5)

# Print the cross-validation scores
print("Cross-Validation Scores:", scores)
print("Mean Score:", scores.mean())

Cross-Validation Scores: [-0.02371579 -0.02268796 -0.02645849 -0.02756956 -0.02651919]
Mean Score: -0.02539019914585037
